### Root Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 1000  # 28  # native word end index

# prefix suffix file
native_word = True # True for adding native word



In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/6-Root Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [8]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]
#disable_list = []

In [9]:
word_select = df_word_select["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [10]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

999

In [11]:
df_word_result = pd.DataFrame(word_list, columns=["word"])
df_word_result

,word
0,musun
1,sensin
2,teşekkürler
3,ara
4,bundan
...,...
994,oraya
995,olun
996,yaptı
997,sessiz


In [12]:
df_word_result = strip(df_word_result)
df_word_result

,word
0,musun
1,sensin
2,teşekkürler
3,ara
4,bundan
...,...
994,oraya
995,olun
996,yaptı
997,sessiz


In [13]:
df_lemma_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Data/{lang_folder.capitalize()}/Turkish_100000_Word_All_Lemma_Result.xlsx")
df_lemma_all

,word,lemma.atis,lemma.boun,lemma.framenet,lemma.imst,lemma,lemma.kenet,lemma.penn,lemma.spacylookup,lemma.tourism
0,bir,bir,bir,bir,bir,bir,bir,bir,bir,bir
1,bu,bu,bu,bu,bu,bu,bu,bu,bu,bu
2,ne,ne,ne,ne,ne,ne,ne,ne,ne,ne
3,ve,ve,ve,ve,ve,ve,ve,ve,ve,ve
4,için,için,için,için,için,da,iç,da,için,için
...,...,...,...,...,...,...,...,...,...,...
99995,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casus,casusa
99996,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku
99997,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırman,tırmandığını
99998,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çöz,çözülemeyecek


In [14]:
df_lemma_all = strip(df_lemma_all)
df_lemma_all

,word,lemma.atis,lemma.boun,lemma.framenet,lemma.imst,lemma,lemma.kenet,lemma.penn,lemma.spacylookup,lemma.tourism
0,bir,bir,bir,bir,bir,bir,bir,bir,bir,bir
1,bu,bu,bu,bu,bu,bu,bu,bu,bu,bu
2,ne,ne,ne,ne,ne,ne,ne,ne,ne,ne
3,ve,ve,ve,ve,ve,ve,ve,ve,ve,ve
4,için,için,için,için,için,da,iç,da,için,için
...,...,...,...,...,...,...,...,...,...,...
99995,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casus,casusa
99996,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku
99997,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırman,tırmandığını
99998,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çöz,çözülemeyecek


In [15]:
for column in df_lemma_all.columns:
    column_nunique = df_lemma_all[f"{column}"].nunique()
    print(f"{column} : {column_nunique}")

word : 100000
lemma.atis : 99386
lemma.boun : 86986
lemma.framenet : 96794
lemma.imst : 93539
lemma : 88593
lemma.kenet : 85433
lemma.penn : 88593
lemma.spacylookup : 28112
lemma.tourism : 100000


In [16]:
#df_lemma_all = df_lemma_all.iloc[word_start:word_end,]

In [17]:
df_lemma_select = df_lemma_all[["word","lemma.spacylookup"]]

In [18]:
df_lemma_select["lemma.spacylookup"].nunique()

28112

In [19]:
df_word_lemma_merge = pd.merge(df_word_result, df_lemma_select, how="left", on="word")
df_word_lemma_merge.drop_duplicates(inplace=True)
df_word_lemma_merge

,word,lemma.spacylookup
0,musun,mu
1,sensin,sen
2,teşekkürler,teşekkür
3,ara,ara
4,bundan,bu
...,...,...
994,oraya,ora
995,olun,ol
996,yaptı,yap
997,sessiz,ses


In [20]:
df_word_lemma_join = df_word_lemma_merge.groupby(["lemma.spacylookup"])["word"].apply(", ".join).reset_index()
df_word_lemma_join

,lemma.spacylookup,word
0,a,a
1,acaba,acaba
2,acele,acele
3,acil,acil
4,acı,acı
...,...,...
542,şey,"şeyin, şeye, şeyleri, şeyi, şeyden, şey, şeyler"
543,şimdi,şimdi
544,şimdiye,şimdiye
545,şu,"şu, şuna, şunu"


In [21]:
lemma_length = len(df_word_lemma_join)

In [22]:
df_word_lemma_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_{lemma_length}_Lemma_And_Join_Words.xlsx", index=False)

#### Copy Move And Delete

In [23]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_{lemma_length}_Lemma_And_Join_Words.xlsx")
output_file

['Turkish_1000_Word_547_Lemma_And_Join_Words.xlsx']

In [24]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [25]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass